# Tehnička dokumentacija: Klasifikacija umjetnina u produkciji
## Inteligentni Informacijski Sustavi
### Ivan Fajta, Leo Maršić

## MLOPS Projekt 

Ova aplikacija koristi CNN za klasifikaciju umjetničkih djela. Implementira se pomoću TensorFlow-a, MLflow-a za praćenje modela i Gradio za korisničko sučelje.
Korisnici mogu klasificirati pojedinačne slike ili grupe slika putem CSV datoteka.


## Podatci

The Tate Collection of Art

- 69 000 podataka o umjetninama
- 1500+ klasa smanjeno na 65 s time da 18 novi klasa se koristi u modelu 
- dodane klase imaju između 50 i 100 primjeraka u bazi podataka
- Kompleksan dataset za klasifikaciju


## Struktura modela

- Postavljanje okruženja za MLflow
- Učitavanje i obrada podataka<br>
&nbsp;&nbsp;&nbsp;&nbsp;filtriranje prema dostupnosti slika i minimalnom broju uzoraka po klasi (≥50).
- Definiranje hijerarhijskih oznaka<br>
&nbsp;&nbsp;&nbsp;&nbsp;Klase umjetničkih slika mapiraju se u šire kategorije (npr. "Graphite on paper" → "Graphite").
- Preuzimanje slika
- Podjela na treniranje i testiranje
- Izrada modela
```python
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.25),

        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.4),

        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_hierarchical_classes, activation='softmax')
    ])
```  

- Treniranje modela
- Evaluacija modela
- Praćenje i zapisivanje rezultata<br>
&nbsp;&nbsp;&nbsp;&nbsp;Model i metapodaci pohranjuju se na MLflow server.

## Gradio

Korisnici mogu klasificirati pojedinačne slike ili grupne slike putem CSV datoteka koje sadrže URL-ove.

### Funkcionalnosti

1. Klasifikacija pojedinačne slike:

    Ulaz: Korisnik učitava jednu sliku.<br>
    Izlaz: Model klasificira sliku u "širu kategoriju" i "specifični medij".

2. Grupna klasifikacija putem CSV datoteka:

    Ulaz: CSV datoteka s stupcima thumbnailUrl (URL slike) i medium (točnu tehniku).<br>
    Izlaz: CSV datoteka s predikcijama i točnošću klasifikacije.

Biblioteke: - TensorFlow, MLflow, Gradio, Pandas, NumPy, Pillow, Requests, Scikit-learn

### Struktura

1. Učitavanje modela
2. Label Encoder
3. Mapiranje
4. Obrada slike
    preprocess - resize
5. Klasifikacija jedne slike

    ```python
    def classify_image(img: Image.Image) -> dict:
        x = preprocess_image(img)
        predictions = loaded_model.predict(x)
        pred_index = np.argmax(predictions, axis=1)[0]
        predicted_label = label_encoder.inverse_transform([pred_index])[0]
        broader_category, specific_medium = predicted_label.split(" > ")
        return broader_category, specific_medium

    ```
6. Grupna klasifikacija putev .csv
7. Korisničko sučelje Gradio


## Rezultati

Prethodni rezultati su bili 68% točnosti na 52 tisuće slika i 47 klasa pomoću ResNet modela, a nakon novih 18 klasa koje sadrže 1293 slike točnost je 71%



![Točnost](testni.png)

                                        precision---recall---f1-score---support

![Classification Report](classifi.png)

![Mlflow](mlflow.png)

Testna .cvs datoteka ima ~700 podataka različitih klasa

![Gradio_single](gradio_single.png)
![Gradio_csv](gradio_batch.png)